<a href="https://colab.research.google.com/github/StepanTita/attattr/blob/master/tune-llm/metrics_hate_explain_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install boto3==1.28.53 transformers ekphrasis torch spacy keras tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 47.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 7.7 MB/s eta 0:00:00


In [2]:
! pip install --upgrade urllib3==2.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 1.6 MB/s eta 0:00:0000:010:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.0.3 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.


In [3]:
! git clone https://github.com/hate-alert/HateXplain.git

Cloning into 'HateXplain'...
remote: Enumerating objects: 414, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 414 (delta 111), reused 106 (delta 99), pack-reused 280
Receiving objects: 100% (414/414), 6.76 MiB | 19.23 MiB/s, done.
Resolving deltas: 100% (229/229), done.


In [11]:
! python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 106.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
import sys

sys.path.append('HateXplain')

In [2]:
import os
import json
import time
import datetime

import more_itertools as mit

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader
from PIL import Image, ImageDraw

from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score

import torch
import torch.nn.functional as F

from HateXplain.TensorDataset.datsetSplitter import createDatasetSplit, encodeData
from HateXplain.TensorDataset.dataLoader import combine_features
from HateXplain.Preprocess.dataCollect import get_annotated_data, get_test_data
from HateXplain.Preprocess.spanMatcher import returnMask

import warnings
warnings.filterwarnings('ignore')

KeyboardInterrupt: 

In [ ]:
def on_gpu(f):
    def wrapper():
        if torch.cuda.is_available():
            return f()
        else:
            print('cuda unavailable')

    return wrapper

In [ ]:
if torch.cuda.is_available():
    ! pip install pynvml
    from pynvml import *
    from numba import cuda


@on_gpu
def print_gpu_utilization():
    try:
        nvmlInit()
        handle = nvmlDeviceGetHandleByIndex(0)
        info = nvmlDeviceGetMemoryInfo(handle)
        print(f"GPU memory occupied: {info.used // 1024 ** 2} MB.")
    except Exception as e:
        print(e)


@on_gpu
def free_gpu_cache():
    print("Initial GPU Usage")
    print_gpu_utilization()

    torch.cuda.empty_cache()

    print("GPU Usage after emptying the cache")
    print_gpu_utilization()

In [ ]:
print_gpu_utilization()
free_gpu_cache()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()

In [7]:
MODEL_NAME = 'bert-base-uncased'

DATASET_NAME = 'hatexplain'

NUM_LABELS = 3
N_LATENT = 3

NUM_EPOCHS = 2
BATCH_SIZE = 16

MAX_SEQ_LEN = 128

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [9]:
params = {
    'num_classes': 3,
    'data_file': 'HateXplain/Data/dataset.json',
    'class_label': 'HateXplain/Data/classes.npy',
    'class_names': ['hatespeech', 'normal', 'offensive'],  # ['NORMAL', 'ABUSIVE'],
    'bert_tokens': True,
    'include_special': False,

    'type_attention': 'softmax',
    'max_length': MAX_SEQ_LEN,
    'variance': 1,
    'decay': False,
    'majority': 2,
}

# Make more data

In [10]:
# ! mkdir Data
# ! cp HateXplain/Data/* Data/

In [11]:
annotated_data = get_annotated_data(params)
annotated_data

,post_id,text,annotatorid1,target1,label1,annotatorid2,target2,label2,annotatorid3,target3,label3,rationales,final_label
0,1179055004553900032_twitter,"[i, dont, think, im, getting, my, baby, them, ...",1,[None],normal,2,[None],normal,3,[None],normal,[],normal
1,1179063826874032128_twitter,"[we, cannot, continue, calling, ourselves, fem...",1,[None],normal,2,[None],normal,3,[None],normal,[],normal
2,1178793830532956161_twitter,"[nawt, yall, niggers, ignoring, me]",4,[African],normal,2,[None],normal,3,[African],hatespeech,[],normal
3,1179088797964763136_twitter,"[<user>, i, am, bit, confused, coz, chinese, p...",1,[Asian],hatespeech,4,[Asian],offensive,3,[Asian],hatespeech,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech
4,1179085312976445440_twitter,"[this, bitch, in, whataburger, eating, a, burg...",4,"[Caucasian, Women]",hatespeech,2,"[Women, Caucasian]",hatespeech,3,"[Women, Caucasian]",offensive,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"[if, ur, still, on, twitter, tell, carlton, i,...",217,"[Men, Women, Other]",offensive,199,[None],offensive,215,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,...",offensive
20144,9990225_gab,"[when, i, first, got, on, here, and, said, i, ...",220,[African],offensive,223,"[African, Other]",offensive,231,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...",offensive
20145,9991681_gab,"[was, macht, der, moslem, wenn, der, zion, geg...",206,[Islam],offensive,203,[Other],normal,211,[None],normal,[],normal
20146,9992513_gab,"[it, is, awful, look, at, world, demographics,...",209,[Hispanic],hatespeech,253,[Asian],hatespeech,222,[Asian],offensive,"[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...",hatespeech


In [12]:
def get_data(data):
    post_ids_list = []
    text_list = []
    attention_list = []
    label_list = []

    final_binny_output = []
    print('total_data', len(data))
    for index, row in tqdm(data.iterrows(), total=len(data)):
        annotation = row['final_label']

        text = row['text']
        post_id = row['post_id']
        annotation_list = [row['label1'], row['label2'], row['label3']]
        tokens_all = list(row['text'])

        if annotation != 'undecided':
            tokens_all, attention_masks = returnMask(row, params, tokenizer)
            final_binny_output.append([post_id, annotation, tokens_all, attention_masks, annotation_list])

    return final_binny_output

In [13]:
data = get_data(annotated_data)
data

total_data 20148


  0%|          | 0/20148 [00:00<?, ?it/s]

[['1179055004553900032_twitter',
  'normal',
  [101,
   1045,
   2123,
   2102,
   2228,
   10047,
   2893,
   2026,
   3336,
   2068,
   2317,
   2193,
   2002,
   2038,
   2048,
   2317,
   1046,
   1998,
   18368,
   2015,
   2025,
   2130,
   5028,
   102],
  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
  ['normal', 'normal', 'normal']],
 ['1179063826874032128_twitter',
  'normal',
  [101,
   2057,
   3685,
   3613,
   4214,
   9731,
   10469,
   2015,
   2065,
   1996,
   2916,
   1997,
   2035,
   24185,
   22984,
   2078,
   4995,
   2102,
   8280,
   2748,
   2000,
   1037,
   4424,
   18421,
   2270,
   2862,
   2021,
   2097,
   1037,
   9099,
   11690,
   22437,
   1998,
   19483,
   24185,
   22984,
   2078,
   2022,
   2583,
   2000,
   4607,
   2037,
   2592,
   2006,
   1996,
   7316,
   7

In [14]:
# The post_id_divisions file stores the train, val, test split ids. We select only the test ids.
with open('HateXplain/Data/post_id_divisions.json') as fp:
    id_division = json.load(fp)
id_division

{'test': ['1178610029273976833_twitter',
  '1165785686903009283_twitter',
  '1252707503441313794_twitter',
  '1103385226921762816_twitter',
  '1169443635869487105_twitter',
  '17372469_gab',
  '1168407844095483904_twitter',
  '12643541_gab',
  '24305062_gab',
  '1123306824571158529_twitter',
  '1178857682511511552_twitter',
  '1178929320527462401_twitter',
  '9274864_gab',
  '18943330_gab',
  '1077853_gab',
  '1162547667416899586_twitter',
  '1178978611652710406_twitter',
  '1178412537496383488_twitter',
  '20989384_gab',
  '1083042797995941889_twitter',
  '1079795475560218626_twitter',
  '1108816043865042945_twitter',
  '1112504910485118979_twitter',
  '10682437_gab',
  '1178873120804999169_twitter',
  '13948927_gab',
  '1225858073270476800_twitter',
  '9551353_gab',
  '18529253_gab',
  '1187745116464832515_twitter',
  '1179048983420899328_twitter',
  '13455893_gab',
  '1100389613074558976_twitter',
  '12711894_gab',
  '1179103318762803200_twitter',
  '1179093647867949056_twitter',
  

In [15]:
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]


# Convert dataset into ERASER format: https://github.com/jayded/eraserbenchmark/blob/master/rationale_benchmark/utils.py
def get_evidence(post_id, anno_text, explanations):
    output = []

    indexes = sorted([i for i, each in enumerate(explanations) if each == 1])
    span_list = list(find_ranges(indexes))

    for each in span_list:
        if type(each) == int:
            start = each
            end = each + 1
        elif len(each) == 2:
            start = each[0]
            end = each[1] + 1
        else:
            print('error')

        output.append({"docid": post_id,
                       "end_sentence": -1,
                       "end_token": end,
                       "start_sentence": -1,
                       "start_token": start,
                       "text": ' '.join([str(x) for x in anno_text[start:end]])})
    return output


# To use the metrices defined in ERASER, we will have to convert the dataset
def convert_to_eraser_format(dataset, method, save_split, save_path, id_division):
    final_output = []

    if save_split:
        train_fp = open(save_path + 'train.jsonl', 'w')
        val_fp = open(save_path + 'val.jsonl', 'w')
        test_fp = open(save_path + 'test.jsonl', 'w')

    for tcount, eachrow in enumerate(dataset):
        temp = {}
        post_id = eachrow[0]
        post_class = eachrow[1]
        anno_text_list = eachrow[2]
        majority_label = eachrow[1]

        if majority_label == 'normal':
            continue

        all_labels = eachrow[4]
        explanations = []
        for each_explain in eachrow[3]:
            explanations.append(list(each_explain))

        # For this work, we have considered the union of explanations. Other options could be explored as well.
        if method == 'union':
            final_explanation = [any(each) for each in zip(*explanations)]
            final_explanation = [int(each) for each in final_explanation]

        temp['annotation_id'] = post_id
        temp['classification'] = post_class
        temp['evidences'] = [get_evidence(post_id, list(anno_text_list), final_explanation)]
        temp['query'] = "What is the class?"
        temp['query_type'] = None
        final_output.append(temp)

        if save_split:
            if not os.path.exists(save_path + 'docs'):
                os.makedirs(save_path + 'docs')

            with open(save_path + 'docs/' + post_id, 'w') as fp:
                fp.write(' '.join([str(x) for x in list(anno_text_list)]))

            if post_id in id_division['train']:
                train_fp.write(json.dumps(temp) + '\n')

            elif post_id in id_division['val']:
                val_fp.write(json.dumps(temp) + '\n')

            elif post_id in id_division['test']:
                test_fp.write(json.dumps(temp) + '\n')
            else:
                print(post_id)

    if save_split:
        train_fp.close()
        val_fp.close()
        test_fp.close()

    return final_output

In [16]:
! mkdir -p HateXplain/Data/Evaluation/Model_Eval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
method = 'union'
save_split = True
save_path = 'HateXplain/Data/Evaluation/Model_Eval/'  #The dataset in Eraser Format will be stored here.
convert_to_eraser_format(data, method, save_split, save_path, id_division)

[{'annotation_id': '1179088797964763136_twitter',
  'classification': 'hatespeech',
  'evidences': [[{'docid': '1179088797964763136_twitter',
     'end_sentence': -1,
     'end_token': 21,
     'start_sentence': -1,
     'start_token': 19,
     'text': '19992 24008'}]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '1179085312976445440_twitter',
  'classification': 'hatespeech',
  'evidences': [[{'docid': '1179085312976445440_twitter',
     'end_sentence': -1,
     'end_token': 3,
     'start_sentence': -1,
     'start_token': 2,
     'text': '7743'},
    {'docid': '1179085312976445440_twitter',
     'end_sentence': -1,
     'end_token': 25,
     'start_sentence': -1,
     'start_token': 20,
     'text': '1045 5223 2317 7743 2229'}]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '13851720_gab',
  'classification': 'hatespeech',
  'evidences': [[{'docid': '13851720_gab',
     'end_sentence': -1,
     'end_token': 17,
     'start

In [18]:
test_data = get_test_data(annotated_data, params)
test_data

Loading BERT tokenizer...
total_data 20148


100%|██████████| 20148/20148 [00:59<00:00, 338.37it/s]


,Post_id,Text,Attention,Label
0,1179055004553900032_twitter,"[101, 1045, 2123, 2102, 2228, 10047, 2893, 202...","[0.041666666666666664, 0.041666666666666664, 0...",normal
1,1179063826874032128_twitter,"[101, 2057, 3685, 3613, 4214, 9731, 10469, 201...","[0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.0...",normal
2,1178793830532956161_twitter,"[101, 6583, 26677, 8038, 3363, 9152, 13327, 20...","[0.09090909090909091, 0.09090909090909091, 0.0...",normal
3,1179088797964763136_twitter,"[101, 5310, 1045, 2572, 2978, 5457, 2522, 2480...","[0.0290389020798568, 0.0290389020798568, 0.029...",hatespeech
4,1179085312976445440_twitter,"[101, 2023, 7743, 1999, 2054, 7875, 12514, 212...","[0.029413453722818332, 0.029413453722818332, 0...",hatespeech
...,...,...,...,...
20143,9989999_gab,"[101, 2065, 24471, 2145, 2006, 10474, 2425, 12...","[0.018685979950755202, 0.018685979950755202, 0...",offensive
20144,9990225_gab,"[101, 2043, 1045, 2034, 2288, 2006, 2182, 1998...","[0.026476741382734967, 0.026476741382734967, 0...",offensive
20145,9991681_gab,"[101, 2001, 24532, 2102, 4315, 9587, 25016, 22...","[0.023255813953488372, 0.023255813953488372, 0...",normal
20146,9992513_gab,"[101, 2009, 2003, 9643, 2298, 2012, 2088, 2832...","[0.03505130729982109, 0.03505130729982109, 0.0...",hatespeech


In [19]:
def transform_data(data, max_len):
    def pad_truncate(record):
        out = record
        if len(out) > max_len:
            out = out[:max_len]
        elif len(out) < max_len:
            out = out + [0] * (max_len - len(out))

        return out

    input_ids = []
    segment_ids = []
    # rationale = []
    attention_mask = []
    labels = []
    for e in tqdm(data):
        attention_mask.append(np.array([1] * len(e[1]) + [0] * (max_len - len(e[0]))))
        input_ids.append(np.array(pad_truncate(e[0])))
        segment_ids.append(np.array([0] * max_len))
        # rationale.append(np.array(pad_truncate(e[1])))
        if e[2] == 'hatespeech':
            labels.append(0)
        elif e[2] == 'normal':
            labels.append(1)
        else:
            labels.append(2)
    return {
        'input_ids': torch.tensor(input_ids, dtype=torch.long),
        'token_type_ids': torch.tensor(segment_ids, dtype=torch.long),
        'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
        # 'rationale': torch.tensor(rationale, dtype=torch.float),
        'label_ids': torch.tensor(labels, dtype=torch.long),
    }

In [20]:
val_data = test_data[test_data['Post_id'].isin(id_division['val'])]
test_data = test_data[test_data['Post_id'].isin(id_division['test'])]

In [21]:
test_data

,Post_id,Text,Attention,Label
0,1179055004553900032_twitter,"[101, 1045, 2123, 2102, 2228, 10047, 2893, 202...","[0.041666666666666664, 0.041666666666666664, 0...",normal
5,13851720_gab,"[101, 6874, 8840, 8462, 2099, 15504, 2033, 209...","[0.020105870350018283, 0.020105870350018283, 0...",hatespeech
17,1178818409812746240_twitter,"[101, 5310, 2054, 2106, 1996, 2214, 3203, 2079...","[0.05263157894736842, 0.05263157894736842, 0.0...",normal
25,19346774_gab,"[101, 2004, 2172, 2004, 1045, 9120, 1037, 4289...","[0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.0...",normal
36,1165819983701643266_twitter,"[101, 3348, 2022, 2061, 2204, 1037, 7743, 2022...","[0.07169623374352667, 0.07169623374352667, 0.0...",offensive
...,...,...,...,...
20105,9773208_gab,"[101, 2009, 2467, 2308, 2667, 2023, 4485, 2066...","[0.024039240070025593, 0.033549462131355116, 0...",offensive
20109,9802943_gab,"[101, 2138, 2308, 2052, 2196, 4682, 2055, 2108...","[0.03170778452859637, 0.04425177806006158, 0.0...",offensive
20117,9826963_gab,"[101, 2129, 2003, 2035, 2008, 12476, 5152, 890...","[0.018464095742040013, 0.025768721435570203, 0...",offensive
20128,9867117_gab,"[101, 2092, 2026, 6203, 12010, 4160, 3428, 199...","[0.019737586787124593, 0.019737586787124593, 0...",hatespeech


In [22]:
encoder = LabelEncoder()
encoder.classes_ = np.load('HateXplain/Data/classes.npy')
encoder.inverse_transform([0, 1, 2])

array(['hatespeech', 'normal', 'offensive'], dtype='<U10')

In [23]:
test_final = transform_data(encodeData(test_data, None, params), MAX_SEQ_LEN)
test_final

100%|██████████| 1924/1924 [00:00<00:00, 19996.58it/s]


  0%|          | 0/1924 [00:00<?, ?it/s]

{'input_ids': tensor([[ 101, 1045, 2123,  ...,    0,    0,    0],
         [ 101, 6874, 8840,  ...,    0,    0,    0],
         [ 101, 5310, 2054,  ...,    0,    0,    0],
         ...,
         [ 101, 2129, 2003,  ...,    0,    0,    0],
         [ 101, 2092, 2026,  ...,    0,    0,    0],
         [ 101, 2021, 2009,  ...,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'label_ids': tensor([1, 0, 1,  ..., 2, 0, 1])}

# Generating outputs

In [24]:
def plot_parallel(matrix, tokens):
    # Set figsize
    plt.figure(figsize=(12, 8))

    input_len = len(tokens)

    # Vertical lines for the years
    plt.axvline(x=1, color='black', linestyle='--', linewidth=1)
    plt.axvline(x=5, color='black', linestyle='--', linewidth=1)

    # Add the BEFORE and AFTER
    plt.text(1, input_len + 1, 'A', fontsize=12, color='black', fontweight='bold')
    plt.text(5, input_len + 1, 'B', fontsize=12, color='black', fontweight='bold')

    # Plot the line for each continent
    for i in range(input_len):
        for j in range(input_len):
            # Add the line to the plot
            plt.plot([1, 5], [i, j], marker='o', label='token', color='blue', linewidth=matrix[i][j])

            # Add label of each continent at each year
            plt.text(
                1 - 0.18,  # x-axis position
                i,  # y-axis position
                tokens[i],  # Text
                fontsize=8,  # Text size
                color='black',  # Text color,
            )

            plt.text(
                5 + 0.06,  # x-axis position
                j,  # y-axis position
                tokens[j],  # Text
                fontsize=8,  # Text size
                color='black',  # Text color
            )
        break

    # Add a title ('\n' allow us to jump lines)
    plt.title(f'AttrAttr scores \n\n\n')

    plt.yticks([])  # Remove y-axis
    plt.box(False)  # Remove the bounding box around plot
    plt.show()  # Display the chart

In [25]:
def generate_text_highlighted(preds, tokens, canvas=(450, 300), threshold=0.2):
    im = Image.new(mode='RGBA', size=canvas, color=(255, 255, 255, 255))

    img = ImageDraw.Draw(im)

    def highlight(xy, wh, color):
        img.rectangle(xy=(xy[0] - 0.5, xy[1] - 0.5, xy[0] + wh[0], xy[1] + wh[1]), fill=color)  # , outline='red'

    # highlight((28, 36), wh=(5, 12), color=(255, 0, 0, 100))
    # img.text((28, 36), 'nice Car', fill=(0, 0, 0, 200))

    x = 28
    y = 36
    for i, token in enumerate(tokens):
        if preds[i] >= threshold:
            x += 3
            highlight((x, y), wh=(6 * len(token), 12), color=(255, 0, 0, 100))
            img.text((x, y), token, fill=(0, 0, 0, 200))
        else:
            img.text((x, y), token, fill=(0, 0, 0, 255))

        x += 1.4 * 5 * len(token)
        if x + 35 >= canvas[0]:
            x = 28
            y += 1.4 * 7

    plt.imshow(im)
    plt.show()

    return im

In [26]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [27]:
def return_dataloader(input_ids, token_type_ids, att_masks, labels, batch_size):
    inputs = torch.tensor(input_ids)
    labels = torch.tensor(labels, dtype=torch.long)
    masks = torch.tensor(np.array(att_masks), dtype=torch.uint8)
    data = TensorDataset(inputs, token_type_ids, masks, labels)
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dataloader

In [28]:
def convert_data(test_data, params, list_dict, rational_present=True, topk=2):
    """this converts the data to be with or without the rationals based on the previous predictions"""
    """input: params -- input dict, list_dict -- previous predictions containing rationals
    rational_present -- whether to keep rational only or remove them only
    topk -- how many words to select"""

    temp_dict = {}
    for ele in list_dict:
        temp_dict[ele['annotation_id']] = ele['rationales'][0]['soft_rationale_predictions']

    test_data_modified = []

    for index, row in tqdm(test_data.iterrows(), total=len(test_data)):
        try:
            attention = temp_dict[row['Post_id']]
        except KeyError:
            continue
        topk_indices = sorted(range(len(attention)), key=lambda i: attention[i])[-topk:]
        new_text = []
        new_attention = []
        if rational_present:
            new_attention = [0]
            new_text = [101]
            for i in range(len(row['Text'])):
                if (i in topk_indices):
                    new_text.append(row['Text'][i])
                    new_attention.append(row['Attention'][i])
            new_attention.append(0)
            new_text.append(102)
        else:
            for i in range(len(row['Text'])):
                if i not in topk_indices:
                    new_text.append(row['Text'][i])
                    new_attention.append(row['Attention'][i])
        test_data_modified.append([row['Post_id'], new_text, new_attention, row['Label']])

    df = pd.DataFrame(test_data_modified, columns=test_data.columns)
    return df

In [29]:
def standaloneEval_with_rational(model, test_data, max_len, topk, config):
    test_ready = transform_data(encodeData(test_data, None, params), max_len)

    post_ids = test_data['Post_id'].tolist()

    test_dataloader = return_dataloader(
        test_ready['input_ids'],
        test_ready['token_type_ids'],
        test_ready['attention_mask'],
        test_ready['label_ids'],
        1
    )

    model.eval()

    print("Running eval on test data...")
    t0 = time.time()

    true_labels = []
    pred_labels = []
    logits_all = []
    attention_all = []
    input_mask_all = []

    # Evaluate data for one epoch
    for step, (input_ids, segment_ids, attention_mask, label_ids) in tqdm(enumerate(test_dataloader),
                                                                          total=len(test_dataloader)):

        # Progress update every 10 texts.
        if step % 10 == 0 and step != 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            print()
            print('Elapsed: ', elapsed)

            testf1 = f1_score(true_labels, pred_labels, average='macro')
            testacc = accuracy_score(true_labels, pred_labels)
            testprecision = precision_score(true_labels, pred_labels, average='macro')
            testrecall = recall_score(true_labels, pred_labels, average='macro')

            # Report the final accuracy for this validation run.
            print(' Accuracy: {0:.3f}'.format(testacc))
            print(' F1: {0:.3f}'.format(testf1))
            print(' Precision: {0:.3f}'.format(testprecision))
            print(' Recall: {0:.3f}'.format(testrecall))

        input_ids = input_ids.to(device)
        segment_ids = segment_ids.to(device)
        attention_mask = attention_mask.to(device)
        label_ids = label_ids.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        _, pred, logits = config['preds_from_logits_func'](outputs)

        logits = torch.nn.functional.softmax(logits, dim=-1)

        pred_rationale = config['model_outputs_callback'](outputs).detach().cpu()
        # pred_hard_rationale = (pred_rationale > config['threshold']).long()
        # pred_soft_rationale = exp_distribute_batched(pred_hard_rationale)

        # attr_score should be: (batch_size, sequence_length)

        pred_labels += list(np.argmax(logits.detach().cpu().numpy(), axis=1).flatten())
        true_labels += list(label_ids.detach().cpu().numpy().flatten())
        attention_all += list(pred_rationale)
        input_mask_all += list(attention_mask.detach().cpu().numpy())
        logits_all += list(logits)

        # if step % 100 == 0:
        #     tokens = tokenizer.convert_ids_to_tokens(input_ids[0], skip_special_tokens=False)[:attention_mask[0].sum()]
        #     plot_parallel(pred_rationale, tokens)
        #     generate_text_highlighted(pred_rationale[0], tokens, threshold=0.5)

    attention_vector_final = []
    for x, y in zip(attention_all, input_mask_all):
        temp = []
        for x_ele, y_ele in zip(x, y):
            if y_ele == 1:
                temp.append(x_ele)
        attention_vector_final.append(temp)

    list_dict = []

    for post_id, attention, logits, pred, ground_truth in zip(post_ids, attention_vector_final, logits_all, pred_labels,
                                                              true_labels):
        temp = {}
        encoder = LabelEncoder()
        encoder.classes_ = np.load('HateXplain/Data/classes.npy')
        pred_label = encoder.inverse_transform([pred])[0]
        ground_label = encoder.inverse_transform([ground_truth])[0]

        temp['annotation_id'] = post_id
        temp['classification'] = pred_label
        temp['classification_scores'] = {'hatespeech': logits[0].item(), 'normal': logits[1].item(),
                                         'offensive': logits[2].item()}

        topk_indicies = sorted(range(len(attention)), key=lambda i: attention[i])[-topk:]

        temp_hard_rationales = []
        for ind in topk_indicies:
            temp_hard_rationales.append({'end_token': ind + 1, 'start_token': ind})

        temp['rationales'] = [
            {
                'docid': post_id,
                'hard_rationale_predictions': temp_hard_rationales,
                'soft_rationale_predictions': attention,
                #"soft_sentence_predictions":[1.0],
                'truth': ground_truth
            }
        ]
        list_dict.append(temp)

    return list_dict

In [30]:
def get_final_dict_with_rational(model, test_data, params, max_len, topk=5, config=None):
    list_dict_org = standaloneEval_with_rational(model, test_data, max_len, topk=topk, config=config)
    test_data_with_rational = convert_data(test_data, params, list_dict_org, rational_present=True, topk=topk)

    list_dict_with_rational = standaloneEval_with_rational(model, test_data_with_rational, max_len, topk=topk, config=config)
    test_data_without_rational = convert_data(test_data, params, list_dict_org, rational_present=False, topk=topk)

    list_dict_without_rational = standaloneEval_with_rational(model, test_data_without_rational, max_len, topk=topk, config=config)

    final_list_dict = []
    for ele1, ele2, ele3 in zip(list_dict_org, list_dict_with_rational, list_dict_without_rational):
        ele1['sufficiency_classification_scores'] = ele2['classification_scores']
        ele1['comprehensiveness_classification_scores'] = ele3['classification_scores']
        final_list_dict.append(ele1)
    return final_list_dict

In [31]:
class BERTOutputs:
    def __init__(self, loss, logits, hidden_states):
        self.loss = loss
        self.logits = logits
        self.hidden_states = hidden_states


class BERTwithRationalLossForSequenceClassification(torch.nn.Module):
    def __init__(self, model, rational_weight=0.5, ce_weight=1.0):
        super(BERTwithRationalLossForSequenceClassification, self).__init__()
        self.model = model

        self.ce_weight = ce_weight
        self.rational_weight = rational_weight

        self.classifier = torch.nn.Linear(model.config.hidden_size, model.config.num_labels)

    def to(self, device):
        super().to(device)
        self.device = device
        self.model.to(device)
        return self

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        hidden_states = outputs.hidden_states
        last_hidden_state = hidden_states[-1]  # (B, seq_len, n_embed)

        logits = self.classifier(last_hidden_state)[:, 0, :]

        if labels is not None:
            ce_loss = F.cross_entropy(logits, labels)
            loss = self.ce_weight * ce_loss

            return BERTOutputs(loss, logits, hidden_states)
        else:
            return BERTOutputs(None, logits, hidden_states)

In [32]:
def get_preds_from_logits(outputs):
    probs = F.softmax(outputs.logits, dim=-1).cpu()
    pred = torch.argmax(probs, dim=-1)  # (B)
    return outputs.loss, pred.long(), outputs.logits

In [33]:
def bert_outputs_callback(outputs):
    return F.sigmoid(outputs.hidden_states[-1].mean(dim=-1))

In [34]:
config = {
    'experiment_name': 'default',
    'log_terminal': True,

    'dataset_name': DATASET_NAME,
    'model_name': MODEL_NAME,

    'num_labels': NUM_LABELS,
    'num_epochs': NUM_EPOCHS,
    'iterations': 1,

    'max_seq_len': MAX_SEQ_LEN,
    'batch_size': BATCH_SIZE,

    'threshold': 0.1,

    # funcs:
    'preds_from_logits_func': get_preds_from_logits,
    'model_outputs_callback': bert_outputs_callback,
}

In [36]:
base_name = f'{DATASET_NAME}-{MODEL_NAME.replace("/", "_")}-{NUM_EPOCHS}'
base_name

'hatexplain-bert-base-uncased-2'

In [38]:
# state_dict = torch.load(f'models/{config["experiment_name"]}/{base_name}.bin')
body_bert = AutoModel.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS, output_hidden_states=True)
raw_model = BERTwithRationalLossForSequenceClassification(body_bert)
# raw_model.load_state_dict(state_dict)
raw_model

BERTwithRationalLossForSequenceClassification(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [39]:
raw_model.to(device)

BERTwithRationalLossForSequenceClassification(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [40]:
dict_rational = get_final_dict_with_rational(
    raw_model,
    test_data[test_data['Label'] != 'normal'],
    params,
    max_len=MAX_SEQ_LEN,
    topk=5,
    config=config
)
dict_rational

100%|██████████| 1142/1142 [00:00<00:00, 17943.58it/s]


  0%|          | 0/1142 [00:00<?, ?it/s]

Running eval on test data...


  0%|          | 0/1142 [00:00<?, ?it/s]


Elapsed:  0:00:01
 Accuracy: 0.400
 F1: 0.222
 Precision: 0.167
 Recall: 0.333

Elapsed:  0:00:02
 Accuracy: 0.500
 F1: 0.247
 Precision: 0.222
 Recall: 0.278

Elapsed:  0:00:04
 Accuracy: 0.467
 F1: 0.276
 Precision: 0.383
 Recall: 0.269

Elapsed:  0:00:05
 Accuracy: 0.475
 F1: 0.263
 Precision: 0.306
 Recall: 0.255

Elapsed:  0:00:06
 Accuracy: 0.480
 F1: 0.278
 Precision: 0.312
 Recall: 0.257

Elapsed:  0:00:07
 Accuracy: 0.533
 F1: 0.294
 Precision: 0.333
 Recall: 0.269

Elapsed:  0:00:08
 Accuracy: 0.529
 F1: 0.289
 Precision: 0.337
 Recall: 0.261

Elapsed:  0:00:09
 Accuracy: 0.512
 F1: 0.279
 Precision: 0.324
 Recall: 0.252

Elapsed:  0:00:10
 Accuracy: 0.489
 F1: 0.264
 Precision: 0.297
 Recall: 0.240

Elapsed:  0:00:11
 Accuracy: 0.530
 F1: 0.275
 Precision: 0.309
 Recall: 0.251


KeyboardInterrupt: 

In [ ]:
! mkdir explanations_dicts

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """

    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):  #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:
# for item in dict_rational:
#     for key in ['classification_scores', 'sufficiency_classification_scores', 'comprehensiveness_classification_scores']:
#         for k, v in item[key].items():
#             item[key][k] = v.item()

In [ ]:
path_name_explanation = 'explanations_dicts/bert_top5.json'
with open(path_name_explanation, 'w') as fp:
    fp.write('\n'.join(json.dumps(i, cls=NumpyEncoder) for i in dict_rational))

In [89]:
with open('Data/Evaluation/Model_Eval/test.jsonl', 'r') as fp, open('Data/Evaluation/Model_Eval/test1.jsonl',
                                                                    'w') as out:
    for line in fp.readlines():
        if json.loads(line)['annotation_id'] in list(test_data[test_data['Label'] != 'normal'][:100]['Post_id']):
            out.write(line)

In [90]:
! cd HateXplain/eraserbenchmark/ && PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test --strict --data_dir ../../Data/Evaluation/Model_Eval --results ../../explanations_dicts/bert_top5.json --score_file ../../model_explain_output.json

  1808 MainThread Error in instances: 0 instances fail validation: set()
  2006 MainThread No sentence level predictions detected, skipping sentence-level diagnostic
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to 

In [91]:
with open('model_explain_output.json') as fp:
    output_data = json.load(fp)

print('\nPlausibility')
print('IOU F1 :', output_data['iou_scores'][0]['macro']['f1'])
print('Token F1 :', output_data['token_prf']['instance_macro']['f1'])
print('AUPRC :', output_data['token_soft_metrics']['auprc'])

print('\nFaithfulness')
print('Comprehensiveness :', output_data['classification_scores']['comprehensiveness'])
print('Sufficiency', output_data['classification_scores']['sufficiency'])


Plausibility
IOU F1 : 0.19023168440826554
Token F1 : 0.5037398106909696
AUPRC : 0.6885301529036749

Faithfulness
Comprehensiveness : 0.5338012586906552
Sufficiency 0.0827036264166236
